# 一、神经网络基本原理
## 1.1 神经网络结构：前向传播、反向传播、梯度下降
### 1.1.1 什么是神经网络，其基本结构是？
神经网络启发于生物学的神经元连接，通过大量的模型参数学习数据中的有效特征来完成对应的任务。核心过程包括前向传播和后向传播，前向传播通过设计各种模型结构来更有效的特征提取，后向传播根据损失函数完成能够降低任务损失的参数更新。
### 1.1.2 反向传播算法（Backpropagation）
反向传播是基于链式法则的参数梯度计算方法。前向传播阶段会构建计算图，记录各层的操作与中间结果；输出通过损失函数聚合为一个标量 loss。随后，反向传播沿计算图自后向前递推loss对每层参数的梯度。最终，这些梯度通过梯度下降等优化算法用于更新模型参数。

#### 1. 链式法则
链式法则是用于求复合函数导数的公式，表示复合函数的导数等于各个内部函数导数的乘积。在深度学习中，它是反向传播算法的数学基础，用来将损失函数对输出的梯度，通过层与层之间的依赖关系，一层层地传回去，最终得到每个参数的梯度，从而进行梯度下降优化。

#### 2. 为什么是最后的loss是标量？
因为只有标量才可以有唯一方向降低损失。假设神经网络最终输出是：
- $\hat{y} = f(x; \theta)$
- 损失函数：$L = \text{Loss}(\hat{y}, y)$

需要计算：
$$
\frac{\partial L}{\partial \theta}
$$

这只能在 $L$ 是标量的情况下才能得到明确的导数，保证有唯一方向去更新参数。如果 $L$ 是向量，比如 shape = (batch\_size,)，将无法直接用链式法则去求导。

#### 3. 我理解的误区：
每个样本的loss组成的向量并不是进行反向传播的损失L，我们能够直接根据损失函数求每个样本的导数，但其导数也是对于聚合后的、反向传播的损失L
    
假如定义的 loss 是：

$$
L = \sum_{i=1}^n \frac{1}{2}(y_i - \hat{y}_i)^2
$$

我们想要求：

$$
\frac{\partial L}{\partial \hat{y}_i}
$$

- 人类理解的

    对每一项求导：

    $$
    \frac{\partial L}{\partial \hat{y}_i} = \hat{y}_i - y_i
    $$

    完全正确，没问题！


- 自动求导框架的实际“链式传播路径”

    框架是这样理解你的写法的：

    - **先计算每个样本的损失：**

    $$
    L_i = \frac{1}{2}(y_i - \hat{y}_i)^2
    $$

    - **再通过聚合：**

        - 使用 `sum`：

            $$
            L = \sum_i L_i
            $$

            对 $L_i$ 的导数是 $\frac{\partial L}{\partial L_i} = 1$

    - **再通过链式法则：**

    $$
    \frac{\partial L}{\partial \hat{y}_i} = \frac{\partial L}{\partial L_i} \cdot \frac{\partial L_i}{\partial \hat{y}_i} = 1 \cdot (\hat{y}_i - y_i)
    $$

    所以结果依然是：
       
    $$
    \boxed{\frac{\partial L}{\partial \hat{y}_i} = \hat{y}_i - y_i}
    $$
        
    和直觉上的结果完全一样。

### 1.1.3 梯度下降再认识
#### 1️⃣ 梯度的定义
对标量 loss $L$ 关于参数张量 $W$ 求导：

$$
\nabla_W L = \frac{\partial L}{\partial W}
$$

* 结果是与参数同形状的张量，每个元素对应一个参数的偏导数
* 整个梯度张量在高维参数空间中构成了 loss 上升最快的方向，负梯度方向就是下降最快的方向
* 每个元素 $\frac{\partial L}{\partial W_{ij}}$ 是整个梯度向量的一个分量，loss这个分量下增加的瞬时变化率（即如果只改变该参数一个单位，loss 的变化量）

**一句话总结：** 在深度学习中，loss 是参数张量的函数，其梯度是一个向量，表示当前参数点处 loss 上升最快的方向。每个参数的偏导数（即梯度分量）表示该参数对 loss 的局部影响程度，整体梯度方向指引我们如何更新参数以最快地减小 loss。

**形象比喻：**

想象三维空间的地形（loss landscape）：

* 你站在某一点 $W$ 

* 沿 x、y、z 方向，你能测到坡度（梯度分量）

* 把三个方向的坡度合成一个箭头（梯度向量）

* 顺着负梯度箭头走，就能最快下山（loss下降）


#### 2️⃣ 梯度下降更新
梯度下降是利用梯度信息更新参数，使 loss 下降最快的方法。具体步骤如下：

1. **计算梯度**
   对当前参数 $W$ 计算 loss 的梯度：

   $$
   \nabla_W L = \frac{\partial L}{\partial W}
   $$

2. **沿负梯度方向更新参数**
   使用学习率 $\eta$ 控制步长，更新公式为：

   $$
   W \gets W - \eta \nabla_W L
   $$

   * 这里 $-\nabla_W L$ 是 loss 下降最快的方向。
   * 学习率 $\eta$ 决定每一步走多远，过大可能发散，过小收敛慢。

3. **重复迭代**
   在训练过程中，多次计算梯度并更新参数，逐步减小 loss，直到满足停止条件（如 loss 足够小或达到最大迭代次数）。

#### **形象比喻：**
* 想象你站在三维地形的某一点：

  * 梯度告诉你“地面坡度有多陡、哪个方向坡最大”。
  * 负梯度方向就是下山最快的路。
  * 每次沿这个方向走一步，就相当于更新参数 $W$ 一次。
* 反复多次，就像沿山坡下滑，最终到达低谷（loss 最小）。
---

## 1.2 激活函数

### 1.2.1 什么是激活函数,其作用是？
在神经网络中，**激活函数**是连接在每一层线性变换（如加权求和）之后的**非线性函数**，它的作用包括：

- 引入非线性，使网络可以拟合更复杂的函数关系
- 控制输出的尺度范围，防止数值爆炸
- 有些激活函数还具有稀疏性或抑制特征，有助于正则化。比如relu有稀疏性，把小于0给屏蔽掉；sigmoid有异质性

### 1.2.2 饱和区和敏感区（线性区）
- **饱和区（Saturation Region）：**
  - 输入极大或极小时，**输出趋于常数**，此时梯度（导数）趋近于 0，容易导致 **梯度消失问题**
  - 常见于 Sigmoid、Tanh 等激活函数的两端
  - 表示网络对输入**不敏感**

- **敏感区 / 线性区（Linear/Sensitive Region）：**
  - 激活函数梯度较大、变化快的区域
  - 神经元对输入变化较为敏感，易于学习
  - 例如：Sigmoid 在 \(x=0\) 附近；Tanh 在中心附近

### 1.2.3 常见激活函数

| 函数名 | 表达式 | 输出范围 | 饱和区区间 | 敏感区 | 常用损失函数 | 用途说明 | 优缺点总结 |
|--------|--------|-----------|--------------|------------|----------------|------------|------------|
| **Sigmoid** | $f(x) = \frac{1}{1 + e^{-x}}$ | (0, 1) | $x \ll 0$ 或 $x \gg 0$ | $x \approx 0$ | Binary Cross Entropy | 二分类输出层 | ✅ 输出概率，易理解；❌ 梯度消失，两端饱和，不零中心 |
| **Tanh** | $f(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$ | (−1, 1) | $x \ll 0$ 或 $x \gg 0$ | $x \approx 0$ | MSE 或 BCE | RNN常用，零中心 | ✅ 零中心输出；❌ 梯度消失，饱和区存在 |
| **ReLU** | $f(x) = \max(0, x)$ | [0, ∞) | 无（负区间恒为0） | $x > 0$ | MSE、CE | 默认首选，卷积/全连接 | ✅ 简单高效，梯度不消失；❌ 负区间神经元可能死亡 |
| **Leaky ReLU** | $f(x) = \max(\alpha x, x), \alpha \approx 0.01$ | (−∞, ∞) | 无 | 全区间 | MSE、CE | 缓解死神经元 | ✅ 保持梯度活性，训练鲁棒；❌ 输出非零中心，仍可能过大 |
| **ELU** | $f(x) = \begin{cases} x,& x>0 \\ \alpha (e^x-1),& x\le0 \end{cases}$ | (−α, ∞) | $x \ll 0$ | $x \approx 0$ | MSE、CE | 零中心，训练更平滑 | ✅ 零中心，平滑；❌ 计算稍慢，负区指数计算 |
| **Softmax** | $f(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$ | (0,1), 和为1 | 指数项可能导致放大 | 全维度 | Cross Entropy | 多分类输出层 | ✅ 转化为概率分布；❌ 指数运算可能数值不稳定 |
| **GLU（Gated Linear Unit** | $\text{GLU}(x) = (xW + b) \cdot \sigma(xV + c)$, $\sigma$：Sigmoid 激活函数 | 饱和区间同sigmoid | 敏感区间同sigmoid | \ | 大模型GPT等 | 加入“门控”机制的激活函数，可以学习性地决定保留/抑制哪些特征，但是增加了参数量 |



---

## 1.3 损失函数
### 1.3.1 什么是损失函数及其作用？

损失函数（Loss Function）用于衡量模型输出与真实标签之间的差异，是训练神经网络时优化的目标函数。通过反向传播计算损失关于参数的梯度，引导模型学习更好的参数。

### 1.3.2 常见损失函数

#### 回归任务

| 损失函数        | 适用场景         | 特点说明                   |
| ----------- | ------------ | ---------------------- |
| MSE（均方误差）   | 连续数值预测       | 平滑、凸函数，对异常值敏感          |
| MAE（平均绝对误差） | 连续数值预测       | 对异常值鲁棒，但在零点不可导         |
| Huber Loss  | 连续数值预测，兼顾异常值 | 小误差用 MSE，大误差用 MAE，梯度稳定 |
| MAPE (平均绝对百分比误差) | 任何需要误差占比评价的回归任务 | 误差占真实值比例，适合衡量不同量级的数据,单位无关|

- **MSE**

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

- **MAE**

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

- **Huber Loss**

$$
L_\delta(y, \hat{y}) = 
\begin{cases}
\frac{1}{2}(y - \hat{y})^2 & |y - \hat{y}| \le \delta \\
\delta(|y - \hat{y}| - \frac{1}{2}\delta) & \text{otherwise}
\end{cases}
$$

- **MAPE**
$$
\text{MAPE} = \frac{100\%}{n} \sum_{i=1}^{n} \left| \frac{y_i - \hat{y}_i}{y_i} \right|
$$


#### 分类任务

| 损失函数                             | 适用场景 | 特点说明                      |
| -------------------------------- | ---- | ------------------------- |
| Binary Cross Entropy (BCE)       | 二分类  | 搭配 Sigmoid 输出，错分惩罚大       |
| Categorical Cross Entropy (CCE)  | 多分类  | 搭配 Softmax 输出和 one-hot 标签 |

>说明：
>
> CCE是交叉熵损失函数 + softmax 输出，组合后导数更简洁，有 log-softmax 合并优化

- **BCE**

$$
\text{BCE} = -[y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})]
$$

- **CCE**

$$
\text{CCE} = -\sum_{i=1}^{C} y_i \log(\hat{y}_i)
$$

- **KL 散度（可用于分类/蒸馏）**

$$
D_{\text{KL}}(P \| Q) = \sum_i p_i \log \frac{p_i}{q_i}
$$


#### 表征学习 / 度量学习损失函数

| 损失函数                           | 适用场景                            | 特点说明                                |
| ------------------------------ | ------------------------------- | ----------------------------------- |
| Contrastive Loss               | Siamese 网络，正负样本对                | 拉近正样本，拉远负样本，margin 控制距离             |
| Triplet Loss                   | 三元组（Anchor-Positive-Negative）结构 | 保证锚点到正样本距离小于到负样本距离 + margin         |
| InfoNCE / NT-Xent Loss         | 自监督学习（SimCLR, MoCo 等）           | 一正多负，softmax + 温度系数控制分布尖锐度，支持大批量负样本 |


- **Contrastive Loss**

$$
L = y D^2 + (1 - y) \max(0, m - D)^2
$$

* $D = \| f(x_1) - f(x_2) \|_2$ ：特征空间欧氏距离
* $y \in \{0,1\}$ ：1 表示正样本，0 表示负样本
* $m$ ：margin，负样本希望距离至少大于 $m$


- **Triplet Loss**

$$
L = \max(0, D(a, p) - D(a, n) + m)
$$

* $a$ ：Anchor
* $p$ ：Positive
* $n$ ：Negative
* $D(a,p) = \| f(a) - f(p) \|_2$, $D(a,n) = \| f(a) - f(n) \|_2$
* $m$ ：margin


- **InfoNCE / NT-Xent Loss**

$$
L_i = - \log \frac{\exp(\text{sim}(z_i, z_i^+) / \tau)}{\sum_{j=1}^{2N} \mathbf{1}_{[j \neq i]} \exp(\text{sim}(z_i, z_j) / \tau)}
$$

* $z_i$ ：样本 $i$ 的特征向量
* $z_i^+$ ：正样本特征向量
* $z_j$ ：负样本特征向量
* $\text{sim}(\cdot, \cdot)$ ：余弦相似度
* $\tau$ ：温度系数

---

## 1.4 优化器
## 1.4.1 优化器原理

**定义：**
优化器就是在深度学习反向传播过程中让各个参数 $\theta$ 往正确的方向更新合适的大小的算法，使得更新后的各个参数让损失函数 $L(\theta)$ 值不断逼近全局最小。

**核心思想：**

1. 参数更新遵循梯度下降原则：沿负梯度方向移动，使损失下降。
2. 可引入历史梯度累积（动量）来加速收敛并减少震荡。
3. 学习率 $\eta$ 控制每步更新幅度，可固定或动态调整。
4. 高级优化器（如Adam）结合动量和自适应学习率，实现更稳定收敛。

**动量原理：**

* 直观理解：引入历史梯度后为当前步梯度更新带来了惯性，使其能够减少震荡，跨越小的鞍点
* 动量公式：

$$
v_t = \gamma v_{t-1} + \eta \nabla L(\theta_t), \quad
\theta_{t+1} = \theta_t - v_t
$$

* 含义：当前梯度 + 历史梯度累积 → 平滑参数更新 → 加速下降并减少震荡。


### 1.4.2 常用优化器

| 优化器                | 核心公式 / 思路                                                                                         | 特点                 | 适用场景        |
| ------------------ | ------------------------------------------------------------------------------------------------- | ------------------ | ----------- |
| **SGD**            | $\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)$                                               | 简单、易理解；可能震荡或收敛慢    | 大多数基础任务     |
| **Momentum**       | $v_t = \gamma v_{t-1} + \eta \nabla L$; $\theta_{t+1} = \theta_t - v_t$                           | 平滑更新，收敛更快；减少鞍点震荡   | CNN、RNN     |
| **Nesterov (NAG)** | $v_t = \gamma v_{t-1} + \eta \nabla L(\theta_t - \gamma v_{t-1})$                                 | “提前看”未来梯度；更快收敛     | 深层网络、复杂曲面   |
| **AdaGrad**        | $S_t = S_{t-1} + \nabla L^2$; $\theta_{t+1} = \theta_t - \eta / (\sqrt{S_t} + \epsilon) \nabla L$ | 自适应学习率，稀疏梯度效果好     | NLP、词向量     |
| **RMSProp**        | 类似AdaGrad，但加衰减平均 $\tilde{S}_t = \rho \tilde{S}_{t-1} + (1-\rho) \nabla L^2$                       | 防止学习率过早减小          | RNN、序列模型    |
| **Adam**           | 结合Momentum + RMSProp，$m_t$ 一阶矩，$v_t$ 二阶矩                                                          | 自适应学习率 + 动量，收敛快且稳定 | 绝大多数任务，默认首选 |

### 1.4.3 优化器基础常识
- 最开始是完全梯度下降，但是对于每个样本都要计算梯度和更新导致计算量太大，于是就从减少计算量角度和优化下降路径角度改进
- 从减少计算量角度提出了随机梯度下降，也就是随机抽一个样本进行计算梯度更新，被证明能够非常近似梯度下降。后来，进一步发展为mini-batch，
- 从最优下降路径角度
    - **Momentum动量法**：考虑到历史梯度信息，作为动量加到当前梯度中，通过**加权指数平均法**保证时间越远的梯度影响越小
    - **Nesterov**：提前看未来梯度信息指的是当前参数-历史动量获取下一步的位置，然后求下一步位置的梯度信息，再与历史动量相加，作为最终梯度信息。这样就能根据下一步的梯度信息调整当前参数
    - **AdaGrad**：除了考虑历史梯度信息之外，学习率也应该根据当前梯度进行变化，保证变化幅度快的梯度学习率小，变化慢的梯度学习率大。通过求历史梯度的平方来衡量变化，然后学习率除以根号下历史梯度平方
    - **RMSProp**:AdaGrad的改进，历史越远的梯度应该影响越小，通过**加权指数平均法**实现
    - **Adam**：结合了RMSProp和Momentum动量法

## 1.5 权重初始化
### 1.5.1 什么是权重初始化？
在深度神经网络中，每一层都有可学习的参数，在训练开始的时候需要赋予这些参数初始值，这就是权重初始化。不合理的权重初始化，会导致梯度消失、梯度爆炸和神经网络学习不到有意义的特征（如常数随机初始化），所以在随机初始化的时候应该保证：
- 激活值不过大，也不过小
- 正向传播和反向传播信号不会放大或者衰减
- 神经元有“差异性”，能学习不同特征

### 1.5.2 常见的初始化方法

| 初始化方法     | 初始化公式（正态分布）                                  | 初始化公式（均匀分布）                                    | 适用激活函数        | 特点说明                                 |
|----------------|--------------------------------------------------------|-----------------------------------------------------------|---------------------|------------------------------------------|
| 随机初始化     | $W \sim \mathcal{N}(0, 1)$                              | $W \sim U(-1, 1)$                                          | 通用（但不推荐）    | 简单易用，但容易梯度爆炸或消失           |
| Xavier 初始化  | $W \sim \mathcal{N}(0, \frac{2}{n_{in} + n_{out}})$    | $W \sim U\left(-\sqrt{\frac{6}{n_{in}+n_{out}}}, \sqrt{\frac{6}{n_{in}+n_{out}}}\right)$ | Sigmoid, Tanh       | 保持前向/反向传播方差一致，适合饱和激活  |
| He 初始化       | $W \sim \mathcal{N}(0, \frac{2}{n_{in}})$              | $W \sim U\left(-\sqrt{\frac{6}{n_{in}}}, \sqrt{\frac{6}{n_{in}}}\right)$ | ReLU, Leaky ReLU    | 专为ReLU设计，增强激活有效性，防止死神经元 |
| 常数初始化     | $W = c$                                                | -                                                         | ❌ 不推荐           | 所有神经元行为一致，无法学习差异         |
| 全零初始化     | $W = 0$                                                | -                                                         | ❌ 严重不推荐       | 所有神经元完全一致，训练完全失效         |
> 说明：
>
> * $n_{\text{in}}$：当前层输入神经元个数
> * $n_{\text{out}}$：当前层输出神经元个数
> * 你可以根据需要选用正态或均匀分布，两者都常见

### 1.5.3 深入理解 Xavier 和 He 初始化

Xavier 和 He 初始化的核心目标是：
- 让不同层的激活值和梯度保持方差稳定，从而避免梯度爆炸或消失。
- 它们通过设定参数的初始分布方差，根据每层的输入/输出规模自动调整，来实现不同层激活值方差稳定的

神经网络每一层的输出是：
$$
y = W x + b
$$
**理想情况**是：输出的方差 **不增不减**，也就是说每一层输出值和输入值的方差接近相等。这对于 **前向传播** 和 **反向传播** 都非常关键。

- 🔷 Xavier 初始化（又叫 Glorot 初始化）

    * **设计思想：**

    > 同时考虑输入和输出节点数，保持激活值和梯度的方差在正反向传播中都不发生剧烈变化。

    * **适用函数：** Tanh、Sigmoid 这类激活函数可能饱和，梯度容易消失，因此初始化要温和。

    * **数学依据：**
    如果希望输出方差 ≈ 输入方差：

    $$
    Var(W) = \frac{2}{n_{\text{in}} + n_{\text{out}}}
    $$

    * **效果：**

    * 避免前向传播中激活值太大或太小
    * 避免反向传播中梯度爆炸或消失

- 🔶 He 初始化（又叫 Kaiming 初始化）

    * **设计思想：**

    > ReLU 会把一半的输出砍掉（负的变0），所以必须放大初始权重的方差来补偿这种“信息损失”。

    * **适用函数：** ReLU, Leaky ReLU

    * **数学依据：**
    为了保持激活值方差不变（考虑 ReLU 会减半激活）：

    $$
    Var(W) = \frac{2}{n_{\text{in}}}
    $$

    * **效果：**

    * 更适合 ReLU，输出不易全部变成 0
    * 提高训练初期的信号传播能力

### 1.5.4 为什么初始化为常数或者0不行？
如果 所有权重相等（如常数或0）：
- 所有神经元计算出的值就完全一样
- 反向传播中得到的梯度也完全一样
- 导致神经元“退化”成一样的，没有学习能力
换句话说：初始化相同 → 神经元感受不到差异 → 学不到不同特征 → 网络失效。

### 1.5.5 相邻两层的方差稳定能够通过Xavier和He初始化保证，但是均值又是怎么维持的呢？怎么能够保证前向和后向传播稳定的？

* **结论**：均值由权重均值和输入均值决定，方差由 Xavier/He 初始化保证，二者结合可以让每层输出既接近零中心又稳定。

假设一层神经元输出：

$$
z = \sum_{i=1}^{n} w_i x_i + b
$$

**条件假设**

* 权重 $w_i$ **均值为 0**
* 输入 $x_i$ 均值约为 0
* 权重与输入相互独立
* 偏置 $b$ 初始化为 0 或很小的值

**输出均值分析**

$$
\mathbb{E}[z] = \sum_{i=1}^{n} \mathbb{E}[w_i x_i] + \mathbb{E}[b] = \sum_i \mathbb{E}[w_i]\mathbb{E}[x_i] + 0 = 0
$$

✅ 因此，通过 **权重均值为 0** + **输入均值接近 0**，输出均值自然接近 0，即使层数增加，也不会产生较大偏差。

**输入均值为什么接近 0？**

* 输入层数据通常经过 **标准化（归一化 / 零均值化）**

  * 减去均值，除以标准差
  * 保证第一层输入均值接近 0，方差约为 1
* 对于深层网络，每层权重均值为 0 → 下一层输入均值仍保持在 0 附近
* **因此即使多层堆叠，均值不会偏离太大，保证了网络的稳定性**

**He初始化的些许不同**

He 初始化保证了 ReLU 激活的输出方差稳定，均值从上一层到输出层保持在 0 附近（由上面公式推导）。经过 ReLU 截断后，输出均值略偏正，但仍在 0 附近，加上方差稳定，所以激活值整体仍保持在合理范围，训练过程稳定。

---

## 1.6 梯度消失 / 爆炸 八股整理
### 1.6.1 什么是梯度消失 / 梯度爆炸？

梯度消失与爆炸指的是在**反向传播过程中**，梯度在逐层传递时不断**变小或变大**，导致网络难以有效训练。

* **梯度消失**：
  在反向传播中，梯度在多层网络中逐渐趋近于 0，导致网络前层几乎不更新权重，学习停滞。

* **梯度爆炸**：
  梯度在反向传播中不断放大，最终导致梯度过大，出现权重更新不稳定或溢出，网络无法收敛。

* **梯度消失 / 爆炸的本质原因**：链式法则下梯度连续相乘导致数值不稳定。对于一层网络：

    $$
    \frac{\partial L}{\partial w_i} = \frac{\partial L}{\partial a_L} \cdot \prod_{l=1}^{L} \frac{\partial a_l}{\partial z_l} \cdot \frac{\partial z_l}{\partial w_i}
    $$

    如果每一层的导数（尤其是激活函数导数）都在 $(0,1)$ 之间，那么多个相乘会使梯度快速趋近于 0；反之若都大于 1，则会迅速增大。

###  1.6.2 消除梯度消失 / 爆炸的方法

| 方法                        | 原理               | 说明                        |
| ------------------------- | ---------------- | ------------------------- |
| ✅ 使用 ReLU 或其变种            | 缓解梯度消失           | ReLU 在正区间梯度恒为 1，不会饱和      |
| ✅ Xavier / He 初始化         | 保持前后层激活和梯度方差一致   | 减少梯度失衡的发生                 |
| ✅ 批标准化（BatchNorm）         | 缓解内部协变量偏移，规范激活分布 | 稳定梯度传播                    |
| ✅ 梯度裁剪（Gradient Clipping） | 限制梯度最大值          | 防止梯度爆炸，常用于 RNN            |
| ✅ 使用残差连接（ResNet）          | 减少梯度路径长度         | 缓解深层网络的梯度衰减               |
| ✅ 合理设置学习率和优化器             | 避免训练不稳定          | 例如使用 Adam、RMSprop 等自适应优化器 |

---
## 1.7 标准化、归一化和正则化
### 1.7.1 什么是标准化、归一化和正则化？
- 标准化：将原始的数据变成均值为0，方差为1的分布，该做法会改变数据的原始分布情况 
- 归一化：将数据压缩到固定的范围内，如 [0,1]、[−1,1] 等，该做法不会改变数据的原始分布，但异常点的影响很大。以上两者都能改变量纲带来的影响
- 正则化：通过在损失函数中增加约束项（如 L1/L2）限制模型参数大小，用于降低模型复杂度、防止过拟合，不会改变输入数据分布

### 1.7.2 标准化和归一化的意义？（站在数据预处理角度）

| 作用     | 说明                             | 区别                     |
| ------ | ------------------------------ |----------------------
| 加快收敛速度 | 避免某些特征值太大或太小导致训练不稳定            |标准化是统一分布，数值都变小但是没有固定区间；归一化是固定到特定数值小区间 |
| 提高模型性能 | 标准化有助于梯度方向统一、学习更有效             | 类比在一个“椭圆形碗”里走向最低点（未标准化），优化会在长轴方向来回震荡；标准化后像在圆形碗中，能直接下滑到最低点 |
| 避免特征支配 | 特征尺度差异大会导致权重偏移                 | 两者都对数值尺度进行了减小，不会有明显偏向 |
| 适配某些模型 | SVM、KNN、PCA等对距离或方差敏感的模型需要标准化处理 |\
| 为参数初始化创造良好条件 | 标准化保证输入均值为0，方差为1，有利于 Xavier / He 初始化 的方差保持策略起效;归一化保证输出均值和方差很小来帮助初始化策略起作用 |\|

### 1.7.3 常见标准化和归一化处理

| 方法            | 类型  | 数学公式                                         | 适用场景          | 特点              |
| ------------- | --- | -------------------------------------------- | ------------- | --------------- |
| Min-Max 归一化   | 归一化 | $x' = \frac{x - x_{min}}{x_{max} - x_{min}}$ | 图像像素、范围已知     | 缩放到固定区间，受极端值影响大 |               |                |
| Z-Score 标准化   | 标准化 | $x' = \frac{x - \mu}{\sigma}$                | 近似正态分布的数据     | 输出均值为 0，方差为 1   |
| Max Abs 缩放    | 归一化 | $x' = \frac{x}{\lvert x_{\max} \rvert}$ | 稀疏数据如文本TF-IDF | 保持稀疏性，仅缩放不移动均值 |
| Robust 标准化    | 标准化 | $x' = \frac{x - \text{median}}{\text{IQR}}$ median：数据中位数 IQR=Q3​−Q1：四分位距（中间 50% 数据范围） | 有异常值的数据       | 25% 到 75% 的“中间数据”映射到统一的数值范围内，从而抵抗异常值、保持特征稳定性     |
| log变换 | \ | $x' = \log(x + \epsilon)$ | 数据偏态分布，有异常值 | 对数缩放 → 压缩大数值、缓解偏态分布、减轻异常值影响、提高模型稳定性  |
| L2归一化   | 向量归一化 | $x' = \frac{x}{\|x\|_2}$              | 表征向量归一化 |保持方向不变，单位模长，多用于文本特征、嵌入向量等   |
| L1归一化   | 向量归一化 | $x' = \frac{x}{\|x\|_1}$              | |稀疏建模场景，保证每个样本特征绝对值和为1       |
> 说明：
> * L1归一化和L2归一化实际上是除以对应的范数
> * L1范数：$\|x\|_1 = \sum_i |x_i|$,取向量每个元素的**绝对值之和**
> * L2范数：\|x\|_2 = \sqrt{\sum_i x_i^2}，取向量的 **欧几里得模长**

### 1.7.4 其它
| 方法     | 标准化对象    | 标准化维度    | 公式（核心）        | 主要作用        | 特点   |
| -------- | ------------ | ------------- | ---------------- | --------------- | ------------------ |
| **L1 正则**    | 权重参数 $w$ | 向量整体   | $\lambda \|w\|_1 = \lambda \sum_i \lvert w_i \rvert $  | 控制模型复杂度，稀疏性 | 可使部分权重为 0，有特征选择效果 |
| **L2 正则**  | 权重参数 $w$ | 向量整体    | $\lambda \|w\|_2^2 = \lambda \sum_i w_i^2$  | 控制模型复杂度，防止过拟合 | 平滑约束，保留所有特征        |
| **BatchNorm** | 激活 $x$   | 见说明 | $\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$ | 加速收敛，缓解内部协变量偏移           | 依赖 batch，训练/推理模式不同 |
| **LayerNorm**  | 激活 $x$   | 见说明    | $\hat{x} = \frac{x - \mu_L}{\sqrt{\sigma_L^2 + \epsilon}}$ | 稳定训练，常用于 RNN/Transformer | 不依赖 batch，适合序列模型   |
| **InstanceNorm** | 激活 $x$   |  见说明   | 和 BN 类似，但均值/方差是按单个图像     | 图像风格迁移等     | 保持样本独立性，不依赖 batch  |
> 说明：
> * L1 正则化：导数是 $\text{sign}(w)$（即 $+1$ 或 $-1$），在 $w=0$ 处不可导但有次梯度；需手动添加正则项 $\lambda |w|_1$ 到 loss 中，实现稀疏性。
> * L2 正则化：导数是 $2w$，对所有点可导；通常通过优化器中的 weight_decay 参数自动实现，用于限制权重大小，使训练更稳定。

假设图片为（B，C，W，H），文本为（B，L，D）
- BatchNorm
 - 对于图片来说是不同样本的同一通道的所有（W, H）进行标准化，维度是（N * W * H）
 - 对于文本来说是不同样本，L的相同位置词向量进行标准化, 维度是（N * D）
- LayerNorm
    - 对于图片来说是同一样本的所有通道的所有（W, H）进行标准化，维度是（C * W * H）
    - 对于文本来说是同一样本，每个词向量进行单独标准化，，维度是（1 * D）
- InstanceNorm
    - 对于图片来说每个样本每个通道 独立做标准化，维度是（1 * W * H）
    - 对于文本就是Layernorm，只不过在标准化过程中可学习的仿射参数不同
- LN和IN区别
    - LN和IN都是对[1, 1, D]个变量求均值和方差，得到一个[B, L, 1]形状的均值和方差tensor去做标准化。
    - 两者不同是在仿射变化一步，LN中的仿射变化参数beta和gamma的形状是[1, 1, D]，而IN中是[1, L, 1]

BatchNorm出现的原因
- Internal Covariate Shift 指的是在神经网络训练过程中，由于前层参数更新导致后层输入分布不断变化的问题。这种分布漂移会导致模型训练收敛变慢、训练不稳定。Batch Normalization 的提出正是为了解决这一问题，通过标准化每层的输入来保持稳定分布，提高训练效率。

---

## 1.8 学习率策略（Learning Rate Schedule）八股整理


### 1.8.1 什么是学习率策略？

学习率策略是指**在训练过程中动态调整学习率**的方法，以提升模型训练效果、稳定性和收敛速度。


### 1.8.2 为什么需要学习率策略？

| 原因      | 说明                  |
| ------- | ------------------- |
| 模型训练早期  | 需要较大的学习率以加速收敛       |
| 模型训练中后期 | 需要较小的学习率以细化参数，避免震荡  |
| 动态调整    | 可以更快跳出局部最优、避免训练过早停止 |
| 稳定收敛    | 训练后期防止发散，提高泛化能力     |

### 1.8.3 常见学习率策略

| 策略名称  | 描述  | 公式 / 特点     | 适用场景      |
| --------| --------- | ---------- | ---------- |
| **固定学习率**             | 学习率保持不变                   | $\eta_t = \eta$                                                                      | 简单模型、调试阶段             |
| **Step Decay**        | 每隔固定 epoch 将学习率缩小一倍或乘衰减系数 | $\eta_t = \eta_0 \cdot \gamma^{\lfloor t/s \rfloor}$                                 | 大多数训练任务               |
| **Exponential Decay** | 学习率随时间指数衰减                | $\eta_t = \eta_0 \cdot e^{-\lambda t}$                                               | 更平滑控制收敛过程             |
| **Polynomial Decay**  | 按多项式方式衰减学习率               | $\eta_t = \eta_0 \cdot (1 - \frac{t}{T})^p$                                          | 自定义调整收敛速度             |
| **Cosine Annealing**  | 学习率在训练过程中像余弦函数一样下降        | $\eta_t = \eta_{\min} + \frac{1}{2}(\eta_0 - \eta_{\min})(1 + \cos(\frac{t\pi}{T}))$ | 提高最终收敛稳定性             |
| **Warm-up**           | 初期学习率从很小线性增长              | 逐步增加到目标学习率                                                                           | 防止一开始梯度过大             |
| **Warm-up + Decay**   | 先升后降                      | warmup → constant / decay                                                            | Transformer, BERT 等常用 |
| **Cyclic LR**         | 学习率在区间内周期性变化              | 模拟 SGD 的波动                                                                           | 避免局部最优，提升鲁棒性          |
| **ReduceLROnPlateau** | 验证集指标不提升时降低学习率            | 训练停滞时手动介入                                                                            | Pytorch/Keras 常见用法    |



## 1.9 训练技巧和其它
### 1.9.1 过拟合和欠拟合

### 1.9.2 分类模型和回归模型的区别
输入输出变量均为连续变量的预测问题为回归，输出变量为有限个离散变量的预测问题
为分类问题；回归问题的输出空间是一个定量的空间，分类问题的输出空间是一个定性
的空间 
### 1.9.3 判别模型，生成模型，区别
- P（Y|X）通过X直接得出Y标记，可以直接进行判别，实际上是有了判定边界，从历史
数据学习到模型，结合X样本的特征直接进行计算。 
- P（X,Y）添加了先验概率，需要先求出X和Y的联合概率，再判断最大的概率值对应的
标记，基于（X,Y）的样本分布，选出概率最大的样本对中Y对应的标记 
生成模型带有的信息比判别模型更加丰富，对缺失数据不敏感，计算更简单，可进行增
量学习，但是假设较难成立，由于先验的引入对数据的分布更加敏感，容易产生错误的
分类；判别模型分类边界更灵活，能清晰的分辨出类之间的差异，得到的准确较高，模
型一般属于高方差低偏差的类型，但是无法反映训练数据本身的特性即无法描述整个场
景
### 1.9.4 线性分类算法，非线性分类算法，区别
- 线性分类器：LR,SVM（无核变换或者线性核） 
- 非线性分类算法：决策树，SVM（高斯核），随机森林，GBDT 
线性分类器计算快，编程方便，但是可能欠拟合；非线性分类器编程较麻烦，但是分类
效果更好，不过会有过拟合的现象 
### 1.9.5 为什么我们一般想要变量服从正态分布？
中心极限定理（将大量具有不同分布的随机变量加起来，所得到的新变量将最终具有正
态分布）；给定均值方差，高斯分布最大化信息熵，即保留了最大的不确定性；封闭性，
高斯分布用很多代数运算完的结果还是高斯分布；形式简单，由两个参数确定。
### 1.9.6 特征相关带来的影响 

### 1.9.7 为什么LR权重可以全部初始化为0，NN不行

### 1.9.8 距离度量的方式

### 1.9.9 什么情况下一定会发生过拟合 

### 1.9.10 Add和concat 的区别

### 1.9.11 Softmax的溢出问题
### 1.9.12 Dropout相关

# 二、训练流程八股
## 2.1 数据处理
### 2.1.1 峰度，偏度
- 偏度：统计数据分布偏斜方向和程度的度量，是统计数据分布非对称程度的数字特征（左
正右负） 
- 峰度：表征概率密度分布曲线在平均值处峰值高低的特征数。直观看来，峰度反映了峰
部的尖度 
### 2.1.2 常用的变量变换方法 
### 2.1.3 样本采样方法：
### 2.1.4 样本不均衡如何处理 
### 2.1.5 样本数据缺失如何处理
### 2.1.6 类别特征如何处理
### 2.1.7 白化的概念和意义



## 2.2 构建训练pipelines
## 2.3 模型训练
## 2.4 模型保存与加载
## 2.5 模型评估与上线
期望风险，经验风险，结构风险
K 折交叉验证与留出法对比，精度提升情况

# 三、模型八股
## 3.1 Transformer
### 3.1.1 基本结构与概念
#### ❓ Transformer 的结构包括哪些关键模块？

**答**：Transformer 是一种基于注意力机制的模型架构，主要包括Encoder-Decoder两个模块，两个模块又都包含位置编码、多头注意力机制、前馈网络、残差连接 + LayerNorm，但是Decoder注意力机制是Masked Attention 和 Cross Attention。各个模块具体介绍下：
- Encoder-Decoder 结构：Encoder 编码输入序列，Decoder 生成输出序列。
- 位置编码（Positional Encoding）：补充位置信息，避免模型失去序列感知能力。
- 多头注意力机制（Multi-Head Attention）：并行捕捉不同子空间的信息相关性。
- 前馈网络（Feedforward Layer）：对每个 token 位置独立地进行非线性特征变换。
- 残差连接 + LayerNorm：加速训练，缓解梯度问题，稳定收敛。

---
#### ❓ 为什么 Transformer 放弃了 RNN 架构？

**答**：Transformer 相较于 RNN 具备以下关键优势：
- 全局依赖建模能力：RNN 是逐步传递状态，难以捕捉远距离依赖；Transformer 使用 self-attention 直接连接任意两个 token
- 并行化计算：RNN 结构天然串行，Transformer 可并行处理整个序列，大幅提高训练效率
- 梯度更稳定：没有 RNN 的长路径依赖，避免梯度爆炸/消失

---
#### ❓ 什么是位置编码（Positional Encoding）？为什么需要它？
**答**：由于 Transformer 没有 RNN/CNN 的顺序结构，模型本身不感知位置信息。位置编码为每个 token 加入唯一的位置信号。

常见的两种方式：
- 正余弦编码（sin/cos）：基于不同频率的正弦函数，可泛化到不同长度；
- 可学习编码（learnable embedding）：将位置看作一个 token，一起学习。

目的：引入序列顺序感，使模型能够区分输入中的不同位置。

---
#### ❓ 什么是 Self-Attention？作用是什么？
**答**：Self-Attention 是一种序列内部的信息交互机制，能够根据输入序列中每个位置对其他位置的重要性进行加权汇总。在 Self-Attention 中，序列自身的每个 token 作为 Query、Key、Value，通过打分函数（如点积）决定哪些位置更重要，计算加权平均得到上下文表示。

---
#### ❓ 什么是 Multi-Head Attention？为何使用多头？
**答**：Multi-Head Attention 是将输入分别线性投影到多个不同的子空间，独立计算注意力，然后将多个头的输出拼接起来再做一次线性变换.

多头机制的优势：
- 分散关注点：不同头可以关注输入的不同部分或不同类型的关系；
- 增强表达力：提升模型对复杂模式的建模能力；
- 并行处理：每个注意力头独立计算，提升效率。

---
#### ❓ Feedforward 层的结构和作用？

**答**：Feedforward 层是位置无关的两层前馈网络，对每个 token 独立作用：

$$
\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2
$$

作用：提供非线性变换,提升模型的特征表达能力；

### 3.1.2 核心计算与机制
#### ❓ Attention 的计算公式？

**答**：

$$
\text{Attention}(Q, K, V) = \text{softmax}\left( \frac{QK^T}{\sqrt{d_k}} \right)V
$$

其中：

* $Q$ 是查询向量；
* $K$ 是键向量；
* $V$ 是值向量；
* $d_k$ 是键的维度。

该公式表示：计算查询与键的相关性（点积后缩放），经过 softmax 得到权重，对值向量加权求和，作为最终的注意力输出。

---
#### ❓ 为什么要除以 $\sqrt{d_k}$？

**答**：注意力分数的方差会随着随维度增加而增大，进而得到的注意力分数会出现极端值，导致softmax函数处于饱和区，使得梯度变得非常小，难以通过反向传播有效地训练。缩放有助于维持点积的稳定性，确保梯度在一个合适的范围内。

具体推导见：

<img src="https://pic2.zhimg.com/v2-289d81dcf31aecce0a0b5b29d67a16dd_1440w.jpg" alt="示例图片" width="500" />

---
#### ❓ 多头注意力机制的维度拆分是如何实现的？

**答**：设输入维度为 $d_{model}$，头数为 $h$，则每个头的维度为 $d_k = d_v = d_{model} / h$。

每个头独立投影：

$$
Q_i = XW_i^Q,\quad K_i = XW_i^K,\quad V_i = XW_i^V
$$

计算注意力，最后拼接所有头并线性变换：

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1,...,\text{head}_h)W^O
$$

---
#### ❓ 注意力机制的算法复杂度是多少？

**答**：
* 时间复杂度：$O(n^2 \cdot d)$，其中 $n$ 是序列长度，$d$ 是维度；
* 空间复杂度：同样是 $O(n^2)$，主要来自 $QK^T$ 的 attention matrix。

这也是长序列场景下效率瓶颈的来源。

---
#### ❓ 为什么使用 mask 注意力？怎么实现？

**答**：Mask Attention 是为了解决：
1. Decoder 防止看到未来 token（自回归）；
2. Padding token 不应影响注意力分布。

实现方法是在 attention score 上加上 -∞，使 softmax 后权重为 0：

$$
\text{score}_{ij} = 
\begin{cases}
\text{原始打分}, & \text{位置合法} \\
-\infty, & \text{屏蔽位置}
\end{cases}
$$


---
#### ❓ Transformer 的位置编码如何数学上表示？解决了什么问题？

**答**：位置编码提供序列顺序信息，常见的有正余弦编码：

$$
\text{PE}_{(pos,2i)} = \sin(pos / 10000^{2i/d}) \\
\text{PE}_{(pos,2i+1)} = \cos(pos / 10000^{2i/d})
$$

它允许模型区分不同 token 位置，并支持长度外推。

---
#### ❓ LayerNorm 的数学原理？为什么用？

**答**：LayerNorm 是对每个样本的每个 token 特征向量标准化：

$$
\text{LN}(x) = \gamma \cdot \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

其中：

* $\mu$、$\sigma^2$：均值和方差是对 token 的所有特征维度计算；
* $\gamma, \beta$：可学习的仿射参数（仿射变换）用于恢复表达能力。

Layer Normalization 对每个样本独立进行归一化，适用于序列化数据和变长输入，而 Batch Normalization 在批处理时对特征进行归一化，不适用于序列长度变化的情况。

---
#### ❓ Transformer 输出序列的概率分布是如何计算的？

**答**：Decoder 输出经过一个线性层映射为词表维度，然后过 softmax 得到概率：

$$
P(y_t | y_{<t}, x) = \text{softmax}(W_o h_t)
$$

其中 $h_t$ 是当前时间步输出表示，$W_o$ 是输出投影权重。

---
### 3.1.3训练与优化
#### ❓ Transformer 的损失函数是什么？

**答**：常用的是 **交叉熵损失（Cross Entropy Loss）**，用于序列生成任务中每个 token 的预测概率和真实分布之间的距离：

$$
\mathcal{L} = -\sum_{t=1}^{T} \log P(y_t | y_{<t}, x)
$$

---

#### ❓ 梯度裁剪的数学定义？为什么重要？

**答**：当梯度范数过大（如 L2 范数），可能引发梯度爆炸。梯度裁剪通过如下方式控制：

$$
\text{if } \|\nabla \theta\|_2 > \tau, \quad \nabla \theta \leftarrow \tau \cdot \frac{\nabla \theta}{\|\nabla \theta\|_2}
$$

裁剪后，梯度仍保留方向，但限制其幅度，防止模型不稳定。

---

#### ❓ Transformer 中的学习率预热策略是怎么设计的？

**答**：采用 **Warmup + 衰减** 的策略（如 Vaswani 原始论文）：

$$
lr = d_{model}^{-0.5} \cdot \min(\text{step}^{-0.5}, \text{step} \cdot \text{warmup\_steps}^{-1.5})
$$

* 先缓慢上升；
* 达到峰值后随步数逐渐衰减。

可以防止训练初期权重大幅更新导致模型崩坏。

---

#### ❓ Transformer 的参数共享策略指的是什么？

**答**：在Transformer模型中，特定层（如编码器中的多个相同层）之间或特定操作（如多头注意力中的头）之间共享参数，可以减少模型的总参数量，有助于减轻过拟合。

---

#### ❓ Transformer 如何处理不同长度的输入序列？

**答**：通过 **mask**（padding mask）标记无效位置，使注意力权重忽略它们，同时避免计算冗余。输入仍按最大长度对齐，但实际处理是动态有效的。

---

#### ❓ Transformer 如何优化内存使用处理大数据？

**答**：
* 使用 **梯度累积**（gradient accumulation）；
* 使用 **混合精度训练（fp16）**；
* 使用 **Flash Attention、Sparse Attention** 降低计算/显存；
* 按需裁剪输入（如最长剪裁或 sliding window）。

---

### 四、变种与应用
#### ❓ Transformer 有哪些经典变种？

**答**：

* **BERT**：双向 Encoder-only，用于理解任务；
* **GPT 系列**：Decoder-only，自回归生成；
* **T5**：Encoder-Decoder 同构文本到文本；
* **DeiT / ViT**：图像 Transformer；
* **Swin Transformer**：层次化视觉结构；
* **Longformer / Performer**：处理长序列的稀疏注意力变种。

---

#### ❓ Transformer各个模块有哪些变种？

**答**：参考知乎文章Transformer中的各种改进 - Pikachu5808的文章 - 知乎
https://zhuanlan.zhihu.com/p/651593869

#### ❓ Transformer的位置编码
**答**：参考大语言模型演化史（位置编码篇）：从Sinusoidal到RoPE（一） - Civ的文章 - 知乎
https://zhuanlan.zhihu.com/p/712276260



## 2.2 非神经网络类机器学习模型
### 2.2.1 XGBoost / LightGBM / CatBoost
### 2.2.2 PCA 和SVD区别 
### 2.2.3 k-means 的k怎么取
### 2.2.4 LR，SVM，决策树，NB的区别，联系，优缺点
### 2.2.5 SVM参数调节
### 2.2.6 为什么SVM要转换为对偶问题 
### 2.2.7 libsvm 和 liblinear 有什么区别
### 2.2.8 核函数的应用场景 
### 2.2.9 LR 为什么要用Sigmoid函数，Sigmoid/Softmax函数的优缺点
### 2.2.10 集成学习
- 为什么要用简单的基学习器，不用一个复杂一点的学习器 
### 为什么Adaboost不容易过拟合
stacking/boosting/bagging 的区别 
Random forest，Adaboost, GBDT，XGBoost 的原理，以及常调节参数 
RF 为什么进行列采样
为什么GBDT不擅长处理离散特征，应用时如何处理
LightGBM 如何处理类别特征
XGBoost 中的行抽样，可以起到哪些作用，为什么可以防止过拟合 
XGBoost 步长如何设定 
XGBoost 重要性如何评估
XGBoost 为什么要进行二阶泰勒展开 

# 四、数学基础八股
## 4.1 线性代数
## 4.2 微积分
### 4.2.1 凸函数
#### 凸函数的概念
凸函数是指在其定义域上满足任意两点连线在图像上方的函数，，它的性质是局部最小值必然是全局最小值


- 什么是凸问题
- 常见的凸优化方法


## 4.3 概率论
### 4.3.1 线性代数

## 4.4 优化理论
## 4.5 信息论


## 